In [284]:
!pip install -r requirements.txt

                                              0.0/8.9 MB ? eta -:--:--
                                              0.1/8.9 MB 1.5 MB/s eta 0:00:06
     -                                        0.2/8.9 MB 2.3 MB/s eta 0:00:04
     -                                        0.4/8.9 MB 3.0 MB/s eta 0:00:03
     ---                                      0.7/8.9 MB 3.8 MB/s eta 0:00:03
     -----                                    1.1/8.9 MB 4.7 MB/s eta 0:00:02
     -------                                  1.6/8.9 MB 5.5 MB/s eta 0:00:02
     ---------                                2.0/8.9 MB 6.1 MB/s eta 0:00:02
     -----------                              2.5/8.9 MB 6.5 MB/s eta 0:00:01
     -----------                              2.6/8.9 MB 6.8 MB/s eta 0:00:01
     -----------                              2.6/8.9 MB 6.8 MB/s eta 0:00:01
     -----------                              2.6/8.9 MB 6.8 MB/s eta 0:00:01
     -----------                              2.6/8.9 MB 4.7 MB/s eta 0

In [262]:
import os

os.environ["OPENAI_API_KEY"] = 'Enter your api key here.'

In [263]:
# import pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.vectorstores import Pinecone
# from langchain.chat_models import AzureChatOpenAI
from langchain.chains import RetrievalQA

# load agents and tools modules
import pandas as pd
# from azure.storage.filedatalake import DataLakeServiceClient
from io import StringIO
from langchain.tools.python.tool import PythonAstREPLTool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain import LLMMathChain



from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI

# The easy document loader for text
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.document_loaders import CSVLoader

In [264]:
model_name = 'text-embedding-ada-002'

# initialize embeddings object; for use with user query/input
embed = OpenAIEmbeddings(
                model=model_name
            )

In [265]:
loader = TextLoader('hr_policy.txt')
doc = loader.load()
print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=20)
docs = text_splitter.split_documents(doc)

# initialize langchain vectorstore
vectorstore = FAISS.from_documents(docs, embed)




You have 1 document
You have 29669 characters in that document


In [266]:
# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])
# print([print(x) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters ")

Now you have 93 documents that have an average of 319 characters 


In [267]:
# initialize LLM object
llm = ChatOpenAI(    
    model_name="gpt-3.5-turbo", 
    )

In [268]:
# initialize vectorstore retriever object
timekeeping_policy = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
)

In [269]:
file = "employee_data.csv"
# csv_file = StringIO("employee_data.csv") 
df = pd.read_csv(file) # load employee_data.csv as dataframe
# print(df)
python = PythonAstREPLTool(locals={"df": df}) # set access of python_repl tool to the dataframe


In [270]:
# import pandas as pd
# # Specify the full path of the CSV file
# file = "employee_data.csv"

# # Read the CSV file and create a dataframe using a context manager
# with open(file) as csv_file:
#     df = pd.read_csv(csv_file)
#     # print(df)

# # Initialize the tool with access to the dataframe
# python = PythonAstREPLTool(locals={"df": df})
# # print()

In [271]:
# # loader = CSVLoader(file_path=file,encoding="utf-8")
# loader = CSVLoader(file_path=file, encoding="utf-8", csv_args={
#                 'delimiter': ','})
# data = loader.load()
# print(data)


# vectorstoreCsv = FAISS.from_documents(data, embed)
# # retriever=vectorstore.as_retriever()
# # chain = ConversationalRetrievalChain.from_llm(
# # llm = ChatOpenAI(temperature=0.0,model_name='gpt-3.5-turbo'),
# # retriever=vectorstore.as_retriever())

# python = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=vectorstoreCsv.as_retriever(),
# )

In [272]:
# create calculator tool
calculator = LLMMathChain.from_llm(llm=llm, verbose=True)

In [273]:
# create variables for f strings embedded in the prompts
user = 'Alexander Verdad' # set user
df_columns = df.columns.to_list() # print column names of df
print(df_columns)

['employee_id', 'name', 'position', 'organizational_unit', 'rank', 'hire_date', 'regularization_date', 'vacation_leave', 'sick_leave', 'basic_pay_in_php', 'employment_status', 'supervisor']


In [274]:
# prep the (tk policy) vectordb retriever, the python_repl(with df access) and langchain calculator as tools for the agent
tools = [
    Tool(
        name = "Timekeeping Policies",
        func=timekeeping_policy.run,
        description="""
        Useful for when you need to answer questions about employee timekeeping policies.
        """
    ),
    Tool(
        name = "Employee Data",
        func=python.run,
        description = f"""
        Useful for when you need to answer questions about employee data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: {df_columns}
        
        <user>: How many Sick Leave do I have left?
        <assistant>: df[df['name'] == '{user}']['sick_leave']
        <assistant>: You have n sick leaves left.              
        """
    ),
    Tool(
        name = "Calculator",
        func=calculator.run,
        description = f"""
        Useful when you need to do math operations or arithmetic.
        """
    )
]

In [275]:
# change the value of the prefix argument in the initialize_agent function. This will overwrite the default prompt template of the zero shot agent type
agent_kwargs = {'prefix': f'You are friendly HR assistant. You are tasked to assist the current user: {user} on questions related to HR. You have access to the following tools:'}

In [276]:
# initialize the LLM agent
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True, 
                         agent_kwargs=agent_kwargs
                         )

In [277]:
# define q and a function for frontend
def get_response(user_input):
    response = agent.run(user_input)
    return response

In [278]:
print(agent.agent.llm_chain.prompt.template)

You are friendly HR assistant. You are tasked to assist the current user: Alexander Verdad on questions related to HR. You have access to the following tools:

Timekeeping Policies: 
        Useful for when you need to answer questions about employee timekeeping policies.
        
Employee Data: 
        Useful for when you need to answer questions about employee data stored in pandas dataframe 'df'. 
        Run python pandas operations on 'df' to help you get the right answer.
        'df' has the following columns: ['employee_id', 'name', 'position', 'organizational_unit', 'rank', 'hire_date', 'regularization_date', 'vacation_leave', 'sick_leave', 'basic_pay_in_php', 'employment_status', 'supervisor']
        
        <user>: How many Sick Leave do I have left?
        <assistant>: df[df['name'] == 'Alexander Verdad']['sick_leave']
        <assistant>: You have n sick leaves left.              
        
Calculator: 
        Useful when you need to do math operations or arithmetic.
 

In [279]:
# get_response('What is my name and employee id?')

In [280]:

get_response('Are employees on probation allowed to have vacation leaves?')



> Entering new  chain...
I should check the timekeeping policies to see if probationary employees are allowed to take vacation leaves.
Action: Timekeeping Policies
Action Input: Search for "vacation leaves for probationary employees"
Observation: Based on the context given, there is no information about vacation leaves for probationary employees. The only leaves mentioned for probationary employees are Unpaid Leave, Military Service Leave, Jury Duty/Civic Duty Leave, Personal Leave. However, these are not vacation leaves.
Thought:I should check the employee data to see if Alexander Verdad is on probation.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad']['employment_status']
Observation: 2    Permanent
Name: employment_status, dtype: object
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Alexander Verdad is a permanent employee and not on probation. I can answer the original question.
Final Answer: Yes, Alexander Verdad is allowed to have vacation leaves as a permanent employee.

> Finished chain.


'Yes, Alexander Verdad is allowed to have vacation leaves as a permanent employee.'

In [283]:
get_response("How many vacation leaves do I have left and what is the policy on unused VLs?")



> Entering new  chain...
I need to check the employee data for Alexander Verdad to see how many vacation leaves he has left, and also check the timekeeping policies for the policy on unused VLs.
Action: Employee Data
Action Input: df[df['name'] == 'Alexander Verdad']['vacation_leave']
Observation: 2    45
Name: vacation_leave, dtype: int64
Thought:Now I need to check the timekeeping policies for the policy on unused VLs.
Action: Timekeeping Policies
Action Input: Check the company policy on unused vacation leaves.
Observation: According to the company policy, unused Vacation Leave can be carried over to the next year, but the total accumulated leave should not exceed 30 days. Any excess leave will be forfeited. Unused Vacation Leave can also be encashed at the end of the year at the basic salary rate. However, employees on probation are not eligible for Vacation Leave. Leave applications must be submitted through the Employee Self Service portal at least one day before the intended l

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

So Alexander Verdad has 45 vacation leaves left, and according to the company policy, he can carry over unused leaves up to a maximum of 30 days, and can also choose to encash them at the end of the year at the basic salary rate. 
Final Answer: Alexander Verdad has 45 vacation leaves left and can carry over unused leaves up to a maximum of 30 days, or encash them at the end of the year at the basic salary rate.

> Finished chain.


'Alexander Verdad has 45 vacation leaves left and can carry over unused leaves up to a maximum of 30 days, or encash them at the end of the year at the basic salary rate.'

In [282]:
get_response("How much will I be paid if I encash my unused VLs?")



> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I need to check the timekeeping policies and employee data to calculate the payout.
Action: Timekeeping Policies
Action Input: Check the policy on unused VL payouts

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht


Observation: Unused Vacation Leave can be encashed at the end of the year at the basic salary rate. Encashment rate is basic salary divided by 30 days multiplied by unused leaves to be encashed.
Thought:

...

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

```text
(15000/30)*45
```
...numexpr.evaluate("(15000/30)*45")...

Answer: 22500.0
> Finished chain.

Observation: Answer: 22500.0
Thought:

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-ED7dbl5wnhP3402U5wUi2qw1 on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

I now know the final answer.
Final Answer: If you encash your 45 unused VLs, you will be paid Php 22,500.

> Finished chain.


'If you encash your 45 unused VLs, you will be paid Php 22,500.'